# よく使う操作

In [1]:
# 2系の場合
from __future__ import division, print_function, unicode_literals

## 1. 実験時
### 1.1. 上書き確認

In [2]:
import os

In [3]:
def confirm_overwriting(filename):
    if os.path.exists(filename):
        msg = 'Filename "{}" is already exists. overwrite this?\n[y/n]'
        print(msg.format(filename), end=' ')  # require importing __future__
        while True:
            key = raw_input()
            if key == 'y':
                return
            elif key == 'n':
                raise OSError('Ovarwriting was not allowed.')
            else:
                print('Please answer with "y" or "n".')
                print(msg.format(filename), end=' ')

In [4]:
# ファイルが既に存在している場合，上書きするかどうかをyes, noで確認する．
# もし"n"を押した場合はエラーを出して実験を終了する．
filename = './README.md'  # ※ READMEはこのファイルと同じ階層にあります．
confirm_overwriting(filename)

Filename "./README.md" is already exists. overwrite this?
[y/n] a
Please answer with "y" or "n".
Filename "./README.md" is already exists. overwrite this?
[y/n] b
Please answer with "y" or "n".
Filename "./README.md" is already exists. overwrite this?
[y/n] y


### 1.2. データフレームの作成

In [5]:
import numpy as np
import pandas as pd

In [6]:
def make_dataframe(cond, nrep=1, is_randomized=True):
    cond = {k: np.r_[v,] for k, v in cond.items()}
    index = pd.MultiIndex.from_product(cond.values(), names=cond.keys())
    df = pd.DataFrame(index=index).reset_index()
    df = pd.concat([df for _ in range(nrep)])
    df = df.iloc[np.random.permutation(len(df))] if is_randomized else df
    return df.reset_index(drop=True)

In [7]:
condition = {
    'A': ('a1', 'a2', 'a3'),
    'B': ('b1', 'b2'),
}
df = make_dataframe(condition, nrep=2, is_randomized=False)

### 1.3. 疑似ランダム選択

In [8]:
import numpy as np

In [9]:
def uniform_sample(values, n):
    # なるべく均等にvaluesが割り当てられるようにランダム選択する．
    samples = np.repeat(values, np.floor(n / len(values)))
    samples = np.r_[samples, np.random.choice(
        values, n - len(samples), replace=False)]
    np.random.shuffle(samples)
    return samples

In [10]:
# うち2つは2個取り出され，残りは1個だけ取り出される．
# 'a'だけ4個取り出されるとか，'b'が取り出されない，みたいな事態にはならない．
uniform_sample(('a', 'b', 'c'), 5)

array([u'b', u'a', u'b', u'c', u'a'], 
      dtype='<U1')

## 2. 分析時
### 2.1. 被験者ごとに標準化する

In [11]:
import pandas as pd
from scipy.stats import zscore

In [12]:
df = pd.DataFrame({
    'subj': [1, 1, 1, 1, 1,
             2, 2, 2, 2, 2],
    'RT': [510, 520, 530, 540, 550,
           1010, 1020, 1030, 1040, 1050]
})

df['RT_std'] = df.groupby('subj')['RT'].transform(zscore)
df  # 被験者ごとに個別に反応時間を標準化することができた

,RT,subj,RT_std
0,510,1,-1.414214
1,520,1,-0.707107
2,530,1,0.000000
3,540,1,0.707107
4,550,1,1.414214
5,1010,2,-1.414214
6,1020,2,-0.707107
7,1030,2,0.000000
8,1040,2,0.707107
9,1050,2,1.414214
